In [ ]:
import sys
sys.path.append('/data/ECS/PythonTutorial/')
import log
log.topic_id = '27_OrnekProjeler-DinamikRaporlama'

# Python ile Dinamik Excel Raporlama

Bu projede amaç düzenli olarak hazırlanması gereken bir excel raporunun Python programlama dili ile otomatik hale getirmektir.

Bu projede veri seti olarak Prisma veritabanı üzerindeki müşteri tablosundan faydalanarak günlük olarak yeni segment değişikliği gerçekleşen müşteriler ile ilgili bölge,şube ve portföy bazlı rapor hazırlayacağız.

# Gerekli Kütüphaneler

In [ ]:
import cx_Oracle
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
import sys
import os

# Datanın çekilmesi

Öncelikle işin önemli kısmı olan dataya errişecek sorguyu yazıyoruz

In [ ]:
def connect_db(user_name=None, password=None):
    try:
        with open('/data/home/alperayd/oracle.txt','r') as f:
            r = f.read()
            user_name = r.split(';')[0]
            password = r.split(';')[1]
        
        os.environ['ORACLE_HOME'] = '/data/app/oracle/product/12.2.0/client_1'
        host = "pgarsnb1rw.fw.garanti.com.tr:4520/pgarsnb1b"
        conn = cx_Oracle.connect(user_name, password, host, encoding='UTF-8')
        return conn
    except Exception as ex:
        print(f"bağlantı sağlanamadı, \nHata: {ex}")
        return None
    

In [ ]:
sql = """
    select 
        t1.RECORD_DATE as T1, t2.RECORD_DATE as T2,
        t1.IP_ID,
        t1.CUSTOMER_TP_CODE as LOB_CODE_T1, t2.CUSTOMER_TP_CODE as LOB_CODE_T2,
        t1.CUSTOMER_TP_DESC as LOB_DESC_T1, t2.CUSTOMER_TP_DESC as LOB_DESC_T2,
        t1.BRANCH_NUM       as SUBE_KODU_T1,t2.BRANCH_NUM       as SUBE_KODU_T2,
        t1.BRANCH_NAME      as SUBE_ADI_T1, t2.BRANCH_NAME      as SUBE_ADI_T2,
        t1.BUSINESS_MKT_REG_ORG_NUM  as BOLGE_KODU_T1, t2.BUSINESS_MKT_REG_ORG_NUM as BOLGE_KODU_T2,
        t1.BUSINESS_MKT_REG_ORG_NAME as BOLGE_ADI_T1, t2.BUSINESS_MKT_REG_ORG_NAME as BOLGE_ADI_T2        
    from (
        select 
            t1.RECORD_DATE, IP_ID, CUSTOMER_TP_CODE, CUSTOMER_TP_DESC, 
            BRANCH_NUM, BRANCH_NAME, BUSINESS_MKT_REG_ORG_NUM, BUSINESS_MKT_REG_ORG_NAME
        from pgaredm.v_cnf_customer_hst t1
        left join pgaredm.V_CNF_BRANCH_HST t2 on t1.MAIN_BRANCH_NUM=t2.BRANCH_NUM
        where t1.record_date = to_date('{t1}','YYYY-MM-DD')
            and t2.record_date = to_date('{t1}','YYYY-MM-DD')
    ) t1
    full outer join (
        select 
            t1.RECORD_DATE, IP_ID, CUSTOMER_TP_CODE, CUSTOMER_TP_DESC, 
            BRANCH_NUM, BRANCH_NAME, BUSINESS_MKT_REG_ORG_NUM, BUSINESS_MKT_REG_ORG_NAME
        from pgaredm.v_cnf_customer_hst t1
        left join pgaredm.V_CNF_BRANCH_HST t2 on t1.MAIN_BRANCH_NUM=t2.BRANCH_NUM
        where t1.record_date = to_date('{t2}','YYYY-MM-DD')
            and t2.record_date = to_date('{t1}','YYYY-MM-DD')
    ) t2
    on t1.IP_ID=t2.IP_ID
    where
        (t1.CUSTOMER_TP_CODE!=t2.CUSTOMER_TP_CODE)
        or
        (t1.BRANCH_NUM!=t2.BRANCH_NUM)
"""
sql = sql.format(t1='2022-12-18', t2='2022-11-30')
df = pd.read_sql(sql, connect_db())
print(df.shape)
display(df.head())

# Rapor Oluşturma

## Bölge Raporu

In [ ]:
dfp = pd.pivot_table(df[df.BOLGE_KODU_T1!=df.BOLGE_KODU_T2],
               index='BOLGE_ADI_T1',
               columns='BOLGE_ADI_T2',
               values='IP_ID',
               aggfunc='count'
              )\
    .fillna(0) \
    .astype(int) 

vmax = dfp.max().max()*1.5

dfs = dfp \
    .style \
    .background_gradient(axis=None, vmin=0, vmax=vmax, cmap='Greens' )
display(dfs)

In [ ]:
rapor_path = 'MusteriDevir.xlsx'
dfs.to_excel(rapor_path, sheet_name='Bolge')

In [ ]:
from openpyxl.styles import Alignment

wb = openpyxl.open(rapor_path)
ws = wb['Bolge']
for r in range(1, dfp.shape[1]+2):
    ws.cell(1,r).alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
ws.column_dimensions['A'].width=30
for c in range(1, dfp.shape[0]+2):
    ws.cell(c,1).alignment = Alignment(horizontal='left')
wb.save(rapor_path)

## Şube Raporu

### Verilen Müşteriler

In [ ]:
df[(df.SUBE_KODU_T1!=df.SUBE_KODU_T2) & (df.SUBE_KODU_T2=='37')] \
    .groupby(['BOLGE_KODU_T2', 'BOLGE_ADI_T2', 'SUBE_KODU_T2', 'SUBE_ADI_T2']) \
    .apply(lambda x: pd.Series({
        'BA-': len(x[x.LOB_CODE_T2=='BA']),
        'BB-': len(x[x.LOB_CODE_T2=='BB']),
        'SC-': len(x[x.LOB_CODE_T2=='SC']),
        'SB-': len(x[x.LOB_CODE_T2=='SB']),
        'SA-': len(x[x.LOB_CODE_T2=='SA']),
        'TA-': len(x[x.LOB_CODE_T2=='TA']),
        'K -': len(x[x.LOB_CODE_T2=='K']),
    }))

### Alınan Müşteriler

In [ ]:
df[(df.SUBE_KODU_T1!=df.SUBE_KODU_T2) & (df.SUBE_KODU_T1=='37')] \
    .groupby(['BOLGE_KODU_T1', 'BOLGE_ADI_T1', 'SUBE_KODU_T1', 'SUBE_ADI_T1']) \
    .apply(lambda x: pd.Series({
        'BA+': len(x[x.LOB_CODE_T2=='BA']),
        'BB+': len(x[x.LOB_CODE_T2=='BB']),
        'SC+': len(x[x.LOB_CODE_T2=='SC']),
        'SB+': len(x[x.LOB_CODE_T2=='SB']),
        'SA+': len(x[x.LOB_CODE_T2=='SA']),
        'TA+': len(x[x.LOB_CODE_T2=='TA']),
        'K +': len(x[x.LOB_CODE_T2=='K']),
    }))

### Toplam Alınan/Verilen

In [ ]:
df_verilen = df[(df.SUBE_KODU_T1!=df.SUBE_KODU_T2)] \
    .groupby(['BOLGE_KODU_T2', 'BOLGE_ADI_T2', 'SUBE_KODU_T2', 'SUBE_ADI_T2']) \
    .apply(lambda x: pd.Series({
        'BA-': len(x[x.LOB_CODE_T2=='BA'])*-1,
        'BB-': len(x[x.LOB_CODE_T2=='BB'])*-1,
        'SC-': len(x[x.LOB_CODE_T2=='SC'])*-1,
        'SB-': len(x[x.LOB_CODE_T2=='SB'])*-1,
        'SA-': len(x[x.LOB_CODE_T2=='SA'])*-1,
        'TA-': len(x[x.LOB_CODE_T2=='TA'])*-1,
        'K -': len(x[x.LOB_CODE_T2=='K'])*-1,
    }))

In [ ]:
df_alinan = df[(df.SUBE_KODU_T1!=df.SUBE_KODU_T2)] \
    .groupby(['BOLGE_KODU_T1', 'BOLGE_ADI_T1', 'SUBE_KODU_T1', 'SUBE_ADI_T1']) \
    .apply(lambda x: pd.Series({
        'BA+': len(x[x.LOB_CODE_T2=='BA']),
        'BB+': len(x[x.LOB_CODE_T2=='BB']),
        'SC+': len(x[x.LOB_CODE_T2=='SC']),
        'SB+': len(x[x.LOB_CODE_T2=='SB']),
        'SA+': len(x[x.LOB_CODE_T2=='SA']),
        'TA+': len(x[x.LOB_CODE_T2=='TA']),
        'K +': len(x[x.LOB_CODE_T2=='K']),
    }))

In [ ]:
pd.set_option('max_columns', 22)
df_all = pd.merge(left=df_alinan.reset_index(),
                  right=df_verilen.reset_index(),
                  how='outer',
                  left_on='SUBE_KODU_T1', right_on='SUBE_KODU_T2'
        )
df_all

### Şube Toplamları

In [ ]:
df_all['BOLGE_KODU'] = df_all.apply(lambda x: x.BOLGE_KODU_T2 if pd.isna(x.BOLGE_KODU_T1) else x.BOLGE_KODU_T1, axis=1)
df_all['BOLGE_ADI'] = df_all.apply(lambda x: x.BOLGE_ADI_T2 if pd.isna(x.BOLGE_ADI_T1) else x.BOLGE_ADI_T1, axis=1)
df_all['SUBE_KODU'] = df_all.apply(lambda x: x.SUBE_KODU_T2 if pd.isna(x.SUBE_KODU_T1) else x.SUBE_KODU_T1, axis=1)
df_all['SUBE_ADI'] = df_all.apply(lambda x: x.SUBE_ADI_T2 if pd.isna(x.SUBE_ADI_T1) else x.SUBE_ADI_T1, axis=1)
df_sube = df_all[['BOLGE_KODU','BOLGE_ADI','SUBE_KODU','SUBE_ADI',
                   'BA+','BB+', 'SC+', 'SB+', 'SA+', 'TA+', 'K +',
                   'BA-', 'BB-', 'SC-','SB-', 'SA-', 'TA-', 'K -'
                  ]] \
    .fillna(0)
display(df_sube)

### Bölge Toplamları

Şube tablosundaki şube adı ve şube kodu bilgilerini Toplam olarak güncelliyoruz

In [ ]:
df_bolge = df_all[['BOLGE_KODU','BOLGE_ADI','SUBE_KODU','SUBE_ADI',
                   'BA+','BB+', 'SC+', 'SB+', 'SA+', 'TA+', 'K +',
                   'BA-', 'BB-', 'SC-','SB-', 'SA-', 'TA-', 'K -'
                  ]] \
    .fillna(0) \
    .assign(SUBE_KODU = 'Toplam') \
    .assign(SUBE_ADI = 'Toplam')
df_bolge

### Bölge Şube Gruplama

In [ ]:
pd.set_option('max_rows',250)
df_final = pd.concat([df_sube, df_bolge]) \
    .sort_values(['BOLGE_ADI','SUBE_KODU']) \
    .groupby(['BOLGE_KODU','BOLGE_ADI','SUBE_KODU','SUBE_ADI']) \
    .sum()
df_final.head(40)

### Kaydet

https://stackoverflow.com/questions/20219254/how-to-write-to-an-existing-excel-file-without-overwriting-data-using-pandas

In [ ]:
wb = openpyxl.load_workbook(rapor_path)
writer = pd.ExcelWriter(rapor_path, engine='openpyxl')
writer.book = wb

writer.sheets = dict((ws.title, ws) for ws in wb.worksheets) # openpyxl gereksinimi

df_final.to_excel(writer, sheet_name="Şube", freeze_panes=(1,4))
writer.save()

In [ ]:
from openpyxl.styles import Font, NamedStyle, PatternFill

wb = openpyxl.open(rapor_path)
ws = wb['Şube']
for c in range(1, df_final.shape[1]+1):
    ws.cell(1,c).alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
ws.column_dimensions['B'].width=30
ws.column_dimensions['D'].width=30

# toplam satırları için bold ve arka renk gri

if 'bold_gri' in wb.named_styles:
    del wb._named_styles[ wb.style_names.index('bold_gri')]

bold_gri = NamedStyle(name="bold_gri")

# color: gri:333333, red: ff0000, green:00ff00, blue:0000ff, orange:ffff00
bold_gri.font = Font(color='333333', bold=True)
wb.add_named_style(bold_gri)


for r in range(1, df_final.shape[0]+2):
    
    # tüm satırlardaki B ve D (2,4) sürunların hizalamasını sola dayalı yapıyoruz
    ws.cell(r,2).alignment = Alignment(horizontal='left')
    ws.cell(r,4).alignment = Alignment(horizontal='left')
    
    # tüm satırlardaki E-R (5,18) sütunların hizalamasını center yapıyoruz
    for c in range(5,19):
        ws.cell(r,c).alignment = Alignment(horizontal='center')

    # toplam satırlarının arka planlarını gri yapıyoruz
    if ws.cell(r,4).value=='Toplam':
        for c in range(3,19):
            ws.cell(r,c).fill = PatternFill(start_color="FFFFFF", end_color="BBBBBB", fill_type = "darkTrellis")
            
wb.save(rapor_path)

# Chart Ekle

In [ ]:
df_verilen = df[(df.LOB_DESC_T1!=df.LOB_DESC_T2)] \
    .groupby(['LOB_DESC_T2']) \
    .apply(lambda x: pd.Series({
        'Verilen': len(x)*-1
    }))
df_alinan = df[(df.LOB_DESC_T1!=df.LOB_DESC_T2)] \
    .groupby(['LOB_DESC_T1']) \
    .apply(lambda x: pd.Series({
        'Alınan': len(x) 
    }))

df_all = pd.merge(left=df_alinan.reset_index(),
                  right=df_verilen.reset_index(),
                  how='outer',
                  left_on='LOB_DESC_T1', right_on='LOB_DESC_T2'
        )

df_all['LOB_DESC'] = df_all.apply(lambda x: x.LOB_DESC_T2 if pd.isna(x.LOB_DESC_T1) else x.LOB_DESC_T1, axis=1)
df_final = df_all[['LOB_DESC','Alınan','Verilen'
                  ]] \
    .fillna(0) \
    .set_index(['LOB_DESC']) \
    .astype(int) \
    .assign(Toplam = lambda x: x.sum(axis=1))

df_final = df_final.assign(positive=lambda x: df_final.Toplam>0)

display(df_final)

In [ ]:
df_final.positive.map({True: 'g', False: 'r'})

In [ ]:
fig, ax = plt.subplots(figsize=(8,5), )
plt.tight_layout()
ax.bar(x=df_final.index, height=df_final.Toplam,
       color=df_final.positive.map({True: 'g', False: 'r'})
      )

colors = df_final.positive.map({True: 'g', False: 'r'}).values.tolist()
for rect, label, color in zip(ax.patches, df_final.Toplam, colors):
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width() / 2, 
        0 if color=='r' else -350 , 
        f'{label:,.0f}', 
        ha="center", va="bottom", c=color
    )
[ax.spines[s].set_visible(False) for s in ('left','top','right')]
ax.set_xticks(np.arange(df_final.shape[0]), df_final.index, rotation='45', ha='right')
ax.grid(axis='y')

plt.subplots_adjust(bottom=0.3, top=0.9)
plt.title('İşkolu Değişimleri')

plt.savefig("myplot.png", dpi = 150)

plt.show()


In [ ]:
wb = openpyxl.load_workbook(rapor_path)
ws = wb.create_sheet('İşKolu')

img = openpyxl.drawing.image.Image('myplot.png')

ws.add_image(img, anchor='B3')
wb.save(rapor_path)

# Class Yap

In [ ]:
import cx_Oracle
import pandas as pd
import numpy as np
import openpyxl
import sys


class Raporlar():
    class MusteriDevir():
        def connect_db(user_name=None, password=None):
            try:
                with open('/data/home/alperayd/oracle.txt','r') as f:
                    r = f.read()
                    user_name = r.split(';')[0]
                    password = r.split(';')[1]

                os.environ['ORACLE_HOME'] = '/data/app/oracle/product/12.2.0/client_1'
                host = "pgarsnb1rw.fw.garanti.com.tr:4520/pgarsnb1b"
                conn = cx_Oracle.connect(user_name, password, host, encoding='UTF-8')
                return conn
            except Exception as ex:
                print(f"bağlantı sağlanamadı, \nHata: {ex}")
                return None
            
        def calistir(self, t1, t2, path='MusteriDevir.xlsx'):
            self.rapor_path = path
            self._data_cek(t1=t1, t2=t2)
            self._bolge_raporu()
            self._sube_raporu()
            self._rapor_yap()
            
        def _data_cek(self, t1, t2):
            print('Data çekiliyor')
            sql = """
                select 
                    t1.RECORD_DATE as T1, t2.RECORD_DATE as T2,
                    t1.IP_ID,
                    t1.CUSTOMER_TP_CODE as LOB_CODE_T1, t2.CUSTOMER_TP_CODE as LOB_CODE_T2,
                    t1.CUSTOMER_TP_DESC as LOB_DESC_T1, t2.CUSTOMER_TP_DESC as LOB_DESC_T2,
                    t1.BRANCH_NUM       as SUBE_KODU_T1,t2.BRANCH_NUM       as SUBE_KODU_T2,
                    t1.BRANCH_NAME      as SUBE_ADI_T1, t2.BRANCH_NAME      as SUBE_ADI_T2,
                    t1.BUSINESS_MKT_REG_ORG_NUM  as BOLGE_KODU_T1, t2.BUSINESS_MKT_REG_ORG_NUM as BOLGE_KODU_T2,
                    t1.BUSINESS_MKT_REG_ORG_NAME as BOLGE_ADI_T1, t2.BUSINESS_MKT_REG_ORG_NAME as BOLGE_ADI_T2        
                from (
                    select 
                        t1.RECORD_DATE, IP_ID, CUSTOMER_TP_CODE, CUSTOMER_TP_DESC, 
                        BRANCH_NUM, BRANCH_NAME, BUSINESS_MKT_REG_ORG_NUM, BUSINESS_MKT_REG_ORG_NAME
                    from pgaredm.v_cnf_customer_hst t1
                    left join pgaredm.V_CNF_BRANCH_HST t2 on t1.MAIN_BRANCH_NUM=t2.BRANCH_NUM
                    where t1.record_date = to_date('{t1}','YYYY-MM-DD')
                        and t2.record_date = to_date('{t1}','YYYY-MM-DD')
                ) t1
                full outer join (
                    select 
                        t1.RECORD_DATE, IP_ID, CUSTOMER_TP_CODE, CUSTOMER_TP_DESC, 
                        BRANCH_NUM, BRANCH_NAME, BUSINESS_MKT_REG_ORG_NUM, BUSINESS_MKT_REG_ORG_NAME
                    from pgaredm.v_cnf_customer_hst t1
                    left join pgaredm.V_CNF_BRANCH_HST t2 on t1.MAIN_BRANCH_NUM=t2.BRANCH_NUM
                    where t1.record_date = to_date('{t2}','YYYY-MM-DD')
                        and t2.record_date = to_date('{t1}','YYYY-MM-DD')
                ) t2
                on t1.IP_ID=t2.IP_ID
                where
                    (t1.CUSTOMER_TP_CODE!=t2.CUSTOMER_TP_CODE)
                    or
                    (t1.BRANCH_NUM!=t2.BRANCH_NUM)
            """
            sql = sql.format(t1=t1, t2=t2)
            df = pd.read_sql(sql, self.connect_db())
            print("Data çekildi:", df.shape)
            self.df = df
            
        def _bolge_raporu(self):
            print('Bölge raporu oluşturuluyor')
            df = self.df
            dfp = pd.pivot_table(df[df.BOLGE_KODU_T1!=df.BOLGE_KODU_T2],
                           index='BOLGE_ADI_T1',
                           columns='BOLGE_ADI_T2',
                           values='IP_ID',
                           aggfunc='count'
                          )\
                .fillna(0) \
                .astype(int) 
            dfs = dfp \
                .style \
                .background_gradient()
            self.dfp = dfp
            self.dfs = dfs
            
        def _sube_raporu(self):
            print('Şube raporu oluşturuluyor')
            df = self.df
            df_verilen = df[(df.SUBE_KODU_T1!=df.SUBE_KODU_T2)] \
                .groupby(['BOLGE_KODU_T2', 'BOLGE_ADI_T2', 'SUBE_KODU_T2', 'SUBE_ADI_T2']) \
                .apply(lambda x: pd.Series({
                    'BA-': len(x[x.LOB_CODE_T2=='BA'])*-1,
                    'BB-': len(x[x.LOB_CODE_T2=='BB'])*-1,
                    'SC-': len(x[x.LOB_CODE_T2=='SC'])*-1,
                    'SB-': len(x[x.LOB_CODE_T2=='SB'])*-1,
                    'SA-': len(x[x.LOB_CODE_T2=='SA'])*-1,
                    'TA-': len(x[x.LOB_CODE_T2=='TA'])*-1,
                    'K -': len(x[x.LOB_CODE_T2=='K'])*-1,
                }))
            
            df_alinan = df[(df.SUBE_KODU_T1!=df.SUBE_KODU_T2)] \
                .groupby(['BOLGE_KODU_T1', 'BOLGE_ADI_T1', 'SUBE_KODU_T1', 'SUBE_ADI_T1']) \
                .apply(lambda x: pd.Series({
                    'BA+': len(x[x.LOB_CODE_T2=='BA']),
                    'BB+': len(x[x.LOB_CODE_T2=='BB']),
                    'SC+': len(x[x.LOB_CODE_T2=='SC']),
                    'SB+': len(x[x.LOB_CODE_T2=='SB']),
                    'SA+': len(x[x.LOB_CODE_T2=='SA']),
                    'TA+': len(x[x.LOB_CODE_T2=='TA']),
                    'K +': len(x[x.LOB_CODE_T2=='K']),
                }))
            
            df_all = pd.merge(left=df_alinan.reset_index(),
                              right=df_verilen.reset_index(),
                              how='outer',
                              left_on='SUBE_KODU_T1', right_on='SUBE_KODU_T2'
                    )
            
            # Şube toplamları -------------------------------------------
            df_all['BOLGE_KODU'] = df_all.apply(lambda x: x.BOLGE_KODU_T2 if pd.isna(x.BOLGE_KODU_T1) else x.BOLGE_KODU_T1, axis=1)
            df_all['BOLGE_ADI'] = df_all.apply(lambda x: x.BOLGE_ADI_T2 if pd.isna(x.BOLGE_ADI_T1) else x.BOLGE_ADI_T1, axis=1)
            df_all['SUBE_KODU'] = df_all.apply(lambda x: x.SUBE_KODU_T2 if pd.isna(x.SUBE_KODU_T1) else x.SUBE_KODU_T1, axis=1)
            df_all['SUBE_ADI'] = df_all.apply(lambda x: x.SUBE_ADI_T2 if pd.isna(x.SUBE_ADI_T1) else x.SUBE_ADI_T1, axis=1)
            df_sube = df_all[['BOLGE_KODU','BOLGE_ADI','SUBE_KODU','SUBE_ADI',
                               'BA+','BB+', 'SC+', 'SB+', 'SA+', 'TA+', 'K +',
                               'BA-', 'BB-', 'SC-','SB-', 'SA-', 'TA-', 'K -'
                              ]] \
                .fillna(0)
            
            # Bölge toplamları -------------------------------------------
            df_bolge = df_all[['BOLGE_KODU','BOLGE_ADI','SUBE_KODU','SUBE_ADI',
                               'BA+','BB+', 'SC+', 'SB+', 'SA+', 'TA+', 'K +',
                               'BA-', 'BB-', 'SC-','SB-', 'SA-', 'TA-', 'K -'
                              ]] \
                .fillna(0) \
                .assign(SUBE_KODU = 'Toplam') \
                .assign(SUBE_ADI = 'Toplam')
            
            # Bölge Toplamlarının groupby ile eklenmesi ------------------
            pd.set_option('max_rows',250)
            df_final = pd.concat([df_sube, df_bolge]) \
                .sort_values(['BOLGE_ADI','SUBE_KODU']) \
                .groupby(['BOLGE_KODU','BOLGE_ADI','SUBE_KODU','SUBE_ADI']) \
                .sum()
                     

            self.df_sube = df_final
            return df_final         
        
        def _chart_olustur(self):
            
            print('Chart oluşturuluyor')
            # rapor datasını oluştur -------------------------------------------------------------------------
            df_verilen = df[(df.LOB_DESC_T1!=df.LOB_DESC_T2)] \
                .groupby(['LOB_DESC_T2']) \
                .apply(lambda x: pd.Series({
                    'Verilen': len(x)*-1
                }))
            df_alinan = df[(df.LOB_DESC_T1!=df.LOB_DESC_T2)] \
                .groupby(['LOB_DESC_T1']) \
                .apply(lambda x: pd.Series({
                    'Alınan': len(x) 
                }))

            df_all = pd.merge(left=df_alinan.reset_index(),
                              right=df_verilen.reset_index(),
                              how='outer',
                              left_on='LOB_DESC_T1', right_on='LOB_DESC_T2'
                    )

            df_all['LOB_DESC'] = df_all.apply(lambda x: x.LOB_DESC_T2 if pd.isna(x.LOB_DESC_T1) else x.LOB_DESC_T1, axis=1)
            df_final = df_all[['LOB_DESC','Alınan','Verilen'
                              ]] \
                .fillna(0) \
                .set_index(['LOB_DESC']) \
                .astype(int) \
                .assign(Toplam = lambda x: x.sum(axis=1))
            
            self.df_chart = df_final
            
            # rapor chart'ını oluştur ------------------------------------------------------------------------
            df_final = df_final.assign(positive=lambda x: df_final.Toplam>0)

            fig, ax = plt.subplots(figsize=(8,4), )
            ax.bar(x=df_final.index, height=df_final.Toplam,
                   color=df_final.positive.map({True: 'g', False: 'r'})
                  )

            colors = df_final.positive.map({True: 'white', False: 'red'}).values.tolist()
            for rect, label, color in zip(ax.patches, df_final.Toplam, colors):
                height = rect.get_height()
                ax.text(
                    rect.get_x() + rect.get_width() / 2, 0 , label, ha="center", va="bottom", c=color
                )
            [ax.spines[s].set_visible(False) for s in ('left','top','right')]
            ax.grid(axis='y')
            
            # chart'ı excel'e ekle -----------------------------------------------------------------------------
            plt.savefig("myplot.png", dpi = 150)
            
        def _rapor_yap(self):
            
            print('Bölge Sayfası ekleniyor')
            # Bölge datasından dosya oluştur -------------------------------------------------------------------
            self.dfs.to_excel(self.rapor_path, sheet_name='Bolge')
            
            dfp = self.dfp
            dfs = self.dfs
            
            # Bölge sayfasının formatını düzenle
            wb = openpyxl.open(self.rapor_path)
            ws = wb['Bolge']
            for r in range(1, dfp.shape[1]+1):
                ws.cell(1,r).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
            ws.column_dimensions['A'].width=30
            for c in range(1, dfp.shape[0]+2):
                ws.cell(c,1).alignment = openpyxl.styles.Alignment(horizontal='left')
            wb.save(self.rapor_path)
            
            # Şube datasını eklemek için ExcelWriter ile dosyayı aç ---------------------------------------------
            print('Şube Sayfası ekleniyor')
            wb = openpyxl.load_workbook(self.rapor_path)
            writer = pd.ExcelWriter(self.rapor_path, engine='openpyxl') 
            writer.book = wb

            writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)
            self.df_sube.to_excel(writer, sheet_name="Şube", freeze_panes=(1,4))
            writer.save()
            
            # Şube sayfasını düzenle -----------------------------------------------------------------------------
            wb = openpyxl.open(rapor_path)
            ws = wb['Şube']
            for c in range(1, df_final.shape[1]+1):
                ws.cell(1,c).alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
            ws.column_dimensions['B'].width=30
            ws.column_dimensions['D'].width=30

            # toplam satırları için bold ve arka renk gri
            bold_gri = NamedStyle(name="bold_gri")
            bold_gri.font = Font(color='333333', bold=True)
            #wb.add_named_style(bold_gri)


            for r in range(1, df_final.shape[0]+2):

                # tüm satırlardaki B ve D (2,4) sürunların hizalamasını sola dayalı yapıyoruz
                ws.cell(r,2).alignment = Alignment(horizontal='left')
                ws.cell(r,4).alignment = Alignment(horizontal='left')

                # tüm satırlardaki E-R (5,18) sütunların hizalamasını center yapıyoruz
                for c in range(5,19):
                    ws.cell(r,c).alignment = Alignment(horizontal='center')

                # toplam satırlarının arka planlarını gri yapıyoruz
                if ws.cell(r,4).value=='Toplam':
                    for c in range(3,19):
                        ws.cell(r,c).fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type = "solid")


            # chartı ekle -----------------------------------------------------------------------------------------
            print('İşkolu sayfası ekleniyor')
            ws = wb.create_sheet('İşKolu')

            print('Chart ekleniyor')
            img = openpyxl.drawing.image.Image('myplot.png')

            ws.add_image(img, anchor='B3')
            wb.save(self.rapor_path)   
            print('Rapor kaydedildi')
            
    class KayipMusteriRaporu():
        def calistir():
            pass
        
    class YeniRapor():
        def calistir():
            pass

In [ ]:
clsr = Raporlar()
md = clsr.MusteriDevir()
md.calistir('2022-07-31','2022-07-25')

In [ ]:
clsr = Raporlar()
clsr.MusteriDevir.calistir()